# COVIDvu - US County Data Comparison (2 Sources) <img src='../resources/American-flag.png' align = 'right'>

Ticket #0364 requires a comparison of two sets of data for US COVID-19 Reporting. This is data from CSBS.org and
NY Times. The following information is needed:

1. Number of counties reported in each data set for the time when you analyze them
2. Statistical differences in the total (which is greater?)
3. Per county comparison
4. Per state comparison

---
# County Totals

### Clone the NY Times Data Repo

COVIDvu is poised to use the CSBS data (API available) for US reporting of confirmed cases, however another source was found
where the NY Times is gathering similar data and providing that via Github repo and CSV formatted files. This data is not updated but cummulative (i.e. ever-growing). The CSBS data is JSON and maintains updated data (current date).

In [8]:
%%bash
if [ -d "./covid-19-data" ]
then
  echo "Directory ./covid-19-data exists."
else
  git clone https://github.com/nytimes/covid-19-data.git
fi

Directory ./covid-19-data exists.


### Clone the CSBS Data API Repo (Reference)

In [9]:
%%bash
if [ -d "./coronavirus-tracker-api" ]
then
  echo "Directory ./coronavirus-tracker-api exists."
else
  git clone https://github.com/ExpDev07/coronavirus-tracker-api.git
fi

Directory ./coronavirus-tracker-api exists.


---
# Strategy

To obtain the data required for our analysis, we should get the snapshots (same date) into similar data structures (JSON). 
The NYTimes data is by day and ever-growing. The CSBS API data is near real-time. These differences make it difficult to 
provide an accurate analysis until both sets of data take the same form. First-take on structure would include county, state
(province), cases, and deaths. Date meanings may be very different if not even comparable. Max-cases reported has more meaning.

### DataFrame NYTimes (All Rows)

This is the entire dataset with the fips column removed for clarity. Ultimately the date column will be removed as it
is not really needed for our comparison of near-realtime data.

In [58]:
import json 
import numpy as np
import pandas as pd 
from pandas import json_normalize #package for flattening json in pandas df

nytimes = pd.read_csv ('covid-19-data/us-counties.csv', index_col=False)
nytimes[['date','county','state','cases','deaths']]

,date,county,state,cases,deaths
0,2020-01-21,Snohomish,Washington,1,0
1,2020-01-22,Snohomish,Washington,1,0
2,2020-01-23,Snohomish,Washington,1,0
3,2020-01-24,Cook,Illinois,1,0
4,2020-01-24,Snohomish,Washington,1,0
...,...,...,...,...,...
15831,2020-03-27,Park,Wyoming,1,0
15832,2020-03-27,Sheridan,Wyoming,5,0
15833,2020-03-27,Sweetwater,Wyoming,1,0
15834,2020-03-27,Teton,Wyoming,12,0


### DataFrame CSBS (All Rows)

Before we can render our CSBS dataframe, we need to pull the data locally via API into a JSON file. This is done here:

In [59]:
%%bash
export COUNTY_CASES_US_ENDPOINT="https://coronavirus-tracker-api.herokuapp.com/v2/locations?source=csbs"
echo "request county data from CSBS API at $COUNTY_CASES_US_ENDPOINT"

curl -s "$COUNTY_CASES_US_ENDPOINT" | jq -M '.locations' > "countyraid-csbs.json" 

request county data from CSBS API at https://coronavirus-tracker-api.herokuapp.com/v2/locations?source=csbs


In [60]:
#load json object
with open('countyraid-csbs.json') as f:
    d = json.load(f)

#lets put the data into a pandas df
csbs = json_normalize(d)
csbs.rename(columns={'last_updated':'date','province':'state', 'latest.confirmed':'cases', 'latest.deaths':'deaths'}, inplace=True)
csbs['date'] = pd.to_datetime(csbs['date']).dt.date
csbs.drop_duplicates(subset=['county','state','cases'], keep='last', inplace=True)

csbs[['date','county','state','cases','deaths']]


,date,county,state,cases,deaths
0,2020-03-29,New York,New York,32308,678
1,2020-03-29,Westchester,New York,7875,10
2,2020-03-29,Nassau,New York,5537,34
3,2020-03-29,Suffolk,New York,4138,37
4,2020-03-29,Rockland,New York,1896,8
...,...,...,...,...,...
1938,2020-03-29,Seminole,Oklahoma,1,0
1939,2020-03-29,Texas,Oklahoma,1,0
1940,2020-03-29,Montgomery,Kansas,3,0
1941,2020-03-29,Stafford,Kansas,1,0


### DataFrame NYTimes (Unique Counties)

In [61]:
nytimes.sort_values(by=['state', 'county', 'date'], inplace = True)
nytimes.drop_duplicates(subset=['county','state'], keep='last', inplace=True)
nytimes[['date','county','state','cases','deaths']]

,date,county,state,cases,deaths
14046,2020-03-27,Autauga,Alabama,6,0
14047,2020-03-27,Baldwin,Alabama,5,0
14048,2020-03-27,Blount,Alabama,5,0
14049,2020-03-27,Bullock,Alabama,2,0
14050,2020-03-27,Butler,Alabama,1,0
...,...,...,...,...,...
15831,2020-03-27,Park,Wyoming,1,0
15832,2020-03-27,Sheridan,Wyoming,5,0
15833,2020-03-27,Sweetwater,Wyoming,1,0
15834,2020-03-27,Teton,Wyoming,12,0


---
# Per County Comparison 

In [62]:
nytimes['county'].isin(csbs['county']).value_counts()

True     1724
False      73
Name: county, dtype: int64

In [69]:
c_compare = pd.merge(nytimes, csbs, on=['state','county'], how='inner')
c_compare.sort_values(by=['state','county'], inplace = True)
c_compare.rename(columns={'cases_x':'cases_nytimes','cases_y':'cases_csbs'}, inplace=True)
c_compare['match?'] = np.where(c_compare['cases_nytimes'] == c_compare['cases_csbs'], 'True', 'False')
c_compare['positive?'] = np.where(c_compare['cases_nytimes'] > c_compare['cases_csbs'], 'True', 'False')
c_compare['negative?'] = np.where(c_compare['cases_nytimes'] < c_compare['cases_csbs'], 'True', 'False')
c_compare[['county','state','cases_nytimes','cases_csbs','match?','positive?','negative?']]
#c_compare.to_csv('countyraid-county-compare.csv')

,county,state,cases_nytimes,cases_csbs,match?,positive?,negative?
0,Autauga,Alabama,6,6,True,False,False
1,Baldwin,Alabama,5,14,False,False,True
2,Blount,Alabama,5,5,True,False,False
3,Bullock,Alabama,2,3,False,False,True
4,Butler,Alabama,1,1,True,False,False
...,...,...,...,...,...,...,...
1710,Park,Wyoming,1,1,True,False,False
1711,Sheridan,Wyoming,5,6,False,False,True
1712,Sweetwater,Wyoming,1,1,True,False,False
1713,Teton,Wyoming,12,14,False,False,True


### Percentage that CSBS has more reported cases

In [74]:
s_compare['negative?'].value_counts(normalize=True) * 100

True     96.078431
False     3.921569
Name: negative?, dtype: float64

---
# Per State Comparison

In [24]:
nytimes['state'].isin(csbs['state']).value_counts()

True     1796
False       1
Name: state, dtype: int64

In [72]:
s_compare = pd.merge(nytimes, csbs, on=['state','county'], how='inner')
s_compare.sort_values(by=['state','county','cases_y'], inplace = True)
s_compare.groupby('state')['cases_y'].sum()
s_compare.sort_values(by=['state','cases_y'], inplace = True)
s_compare.drop_duplicates(subset=['state'], keep='last', inplace=True)
s_compare.rename(columns={'cases_x':'cases_nytimes','cases_y':'cases_csbs'}, inplace=True)
#s_compare.size

s_compare['match?'] = np.where(s_compare['cases_nytimes'] == s_compare['cases_csbs'], 'True', 'False')
s_compare['positive?'] = np.where(s_compare['cases_nytimes'] > s_compare['cases_csbs'], 'True', 'False')
s_compare['negative?'] = np.where(s_compare['cases_nytimes'] < s_compare['cases_csbs'], 'True', 'False')
s_compare['total_csbs'] = s_compare['cases_csbs'].sum()
s_compare['total_nytimes'] = s_compare['cases_nytimes'].sum()

s_compare[['state','cases_nytimes','cases_csbs','match?','positive?','negative?', 'total_nytimes','total_csbs']]
#s_compare.to_csv('countyraid-state-compare.csv')

,state,cases_nytimes,cases_csbs,match?,positive?,negative?,total_nytimes,total_csbs
26,Alabama,179,214,False,False,True,25658,31013
53,Alaska,43,49,False,False,True,25658,31013
59,Arizona,399,545,False,False,True,25658,31013
99,Arkansas,89,92,False,False,True,25658,31013
124,California,1465,1818,False,False,True,25658,31013
167,Colorado,312,367,False,False,True,25658,31013
199,Connecticut,752,908,False,False,True,25658,31013
208,Delaware,106,130,False,False,True,25658,31013
210,District of Columbia,304,342,False,False,True,25658,31013
238,Florida,869,1121,False,False,True,25658,31013


### Percentage that CSBS has more reported cases

In [73]:
s_compare['negative?'].value_counts(normalize=True) * 100

True     96.078431
False     3.921569
Name: negative?, dtype: float64

---
# Experiment Results

---
## Conclusions for No. 1

### Number of counties reported in each data set for the time when you analyze them

- NYTimes reporting 1797 US Counties as of 2020-03-27
- CSBS reporting 1875 US Counties as of 2020-03-27

NOTE: Both datasets may have independent cities (e.g. Virginia) or boroughs (e.g. Alaska) that are bordered by county but reporting as an entity that does not reside in a county proper.

## Conclusions for No. 2

The CSBS data contains reporting county entities but these entities are not reported from a county seat but they still
remain a child of the US state. These extra entities may not be official counties but they still require representation. This does not, however prove that the CSBS data is more accurate but it does provide more reported COVID-19 cases confirmed.

## Conclusions for No. 3

During the per county comparison of COVID-19 reported cases, the CSBS data was updated by mistake of running the notebook again against the CSBS API data. This data is more up to date than the data being collected by NYTimes. The NYTimes data is stuck at 2020-03-27, however the CSBS data is being updated every day (current 2020-03-29). 

While the data was at matched by date, the confirmed cases were higher with the CSBS data than the NYTimes collection.

## Conclusions for No. 4

While the per state comparison was matched by at last_updated date (2020-03-27) the overall state confirmed cases of COVID-19
were higher in the CSBS data as well.